In [1]:
import cobra
import pandas as pd
import matplotlib.pyplot as plt

# Read growth rate data

In [2]:
all_mu = pd.read_csv('data/extracted_growth_rates_with_media.csv', index_col=0)
# select area method (we could have used others, but area works well)"
method = "area"
all_mu = all_mu[all_mu['method'].str.contains("area")]
pca_gluc_mu = all_mu[all_mu['medium'] == "PCA-Gluc"].reset_index()
gluc_mu = all_mu[all_mu['medium'] == "Glucose-MOPS"].reset_index()
citr_mu = all_mu[all_mu['medium'] == "Citrat-MOPS"].reset_index()

mean_pca_gluc_mu = float(pca_gluc_mu['mu'].mean())
std_pca_gluc_mu = float(pca_gluc_mu['mu'].std())
print('PCA-Gluc mu:', mean_pca_gluc_mu, '+=', std_pca_gluc_mu)

mean_gluc_mu = float(gluc_mu['mu'].mean())
std_gluc_mu = float(gluc_mu['mu'].std())
print('glucose-MOPS mu:', mean_gluc_mu, '+=', std_gluc_mu)

mean_citr_mu = float(citr_mu['mu'].mean())
std_citr_mu = float(citr_mu['mu'].std())
print('citrat-MOPS mu:', mean_citr_mu, '+=', std_citr_mu)

PCA-Gluc mu: 0.1557936152166388 += 0.012774444510135584
glucose-MOPS mu: 0.5829104732740061 += 0.022196212314551926
citrat-MOPS mu: 0.7229719008169772 += 0.04890741496672533


In [3]:
means = {
    "PCA-Gluc": mean_pca_gluc_mu,
    "glucose-MOPS": mean_gluc_mu,
    "citrat-MOPS": mean_citr_mu
}

stds = {    
    "PCA-Gluc": std_pca_gluc_mu,
    "glucose-MOPS": std_gluc_mu,
    "citrat-MOPS": std_citr_mu
}

models = {
    "PCA-Gluc": "models/iEZ481_PCA_Gluc.xml",
    "glucose-MOPS": "models/iEZ481_Glucose-MOPS.xml",
    "citrat-MOPS": "models/iEZ481_Citrat-MOPS.xml",
}

objectives = {
    "PCA-Gluc": "EX_glc_e",
    "glucose-MOPS": "EX_glc_e",
    "citrat-MOPS": "EX_cit_e",
}

n_stds = ["-3", "-2", "-1", "+0", "+1", "+2", "+3"]

results = {}

In [4]:
for case in models.keys():
    print('evaluating', case)
    model = cobra.io.read_sbml_model(models[case])
    model.solver = 'gurobi'
    # max minimizes uptake, because signs are flipped for uptakes.
    model.objective.direction = "max"
    model.objective = 'biomass_a'

    fluxes = []

    for std in n_stds:
        # trick to circumvent warning
        model.objective
        model.reactions.get_by_id('biomass_a').upper_bound = 1e6
        model.reactions.get_by_id('biomass_a').lower_bound = -1e6
        model.reactions.get_by_id('biomass_a').lower_bound = means[case] + int(std)*stds[case]
        model.reactions.get_by_id('biomass_a').upper_bound = means[case] + int(std)*stds[case]
        solution = model.optimize()
        # print('for mean +', std,'std we obtain:')
        # print(model.summary())
        assert solution.status == 'optimal'
        fluxes.append(solution.fluxes)
        fluxes[-1].rename("mean" + str(std) + "stds", inplace=True)


    fluxes = pd.concat(fluxes, axis=1)
    fluxes.to_csv(f'data/fba_results/{case}_fluxes.csv')
    print(case)
    print(fluxes)

evaluating PCA-Gluc
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-22
PCA-Gluc
         mean-3stds  mean-2stds  mean-1stds  mean+0stds  mean+1stds  \
glk        0.000000    0.000000    0.000000    0.000000    0.000000   
pgi       -0.149472   -0.165727   -0.181981   -0.198236   -0.214490   
pfkA       0.000000    0.000000    0.000000    0.000000    0.000000   
fbp        0.146669    0.162619    0.178569    0.194518    0.210468   
fda       -0.146669   -0.162619   -0.178569   -0.194518   -0.210468   
...             ...         ...         ...         ...         ...   
EX_C160    0.000000    0.000000    0.000000    0.000000    0.000000   
EX_C180    0.000000    0.000000    0.000000    0.000000    0.000000   
EX_C161    0.000000    0.000000    0.000000    0.000000    0.000000   
EX_C181    0.000000    0.000000    0.000000    0.000000    0.000000   
quiC       0.000000    0.000000    0.000000    0.000000    0.000000   

         mean+2stds  mean+3